In [ ]:
import pandas
import mysql.connector
import pandas 
import numpy as np

import plotly.express as px
import plotly.graph_objects as go


In [ ]:
# Connection à la base

conn = mysql.connector.connect(
    host="192.168.71.42",
    database="myImagiX",
    user="root",
    password="fskfes" )



In [ ]:
def read_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute( query )
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        return pandas.DataFrame( rows, columns=names)
    finally:
        if cursor is not None:
            cursor.close() 


# Piscine d'Amaury

In [ ]:
qr = "SELECT date,state,stateLabel FROM `box_events` LEFT JOIN event_types ON event_types.state = box_events.paramNew WHERE box_events.box_id = 182 AND box_events.date >= '2022-05-01 00:00:00' and event_types.id = box_events.eventId AND idLabel LIKE 'Changement d’état de la filtration' ORDER BY `box_events`.`date` ASC "

dff=read_query(conn,qr)
dff

In [9]:
def transform(x,a,b):
    
    return ((x - np.min(x))/(np.max(x)-np.min(x))) * (b-a) + a

dff['normalised_state']= transform(dff['state'],0,50)

dff.head(10)



,date,state,stateLabel,normalised_state
0,2022-05-01 03:31:04,2,2 Marche manuel ON,32.142857
1,2022-05-01 09:25:53,-2,-2 Arrêt manuel OFF,17.857143
2,2022-05-01 09:29:06,2,2 Marche manuel ON,32.142857
3,2022-05-01 09:44:29,-2,-2 Arrêt manuel OFF,17.857143
4,2022-05-01 09:49:09,2,2 Marche manuel ON,32.142857
5,2022-05-01 18:57:35,0,0 Arrêt AUTO,25.000000
6,2022-05-02 07:00:00,1,1 Marche AUTO,28.571429
7,2022-05-02 10:00:00,0,0 Arrêt AUTO,25.000000
8,2022-05-02 12:00:00,1,1 Marche AUTO,28.571429
9,2022-05-02 15:00:00,0,0 Arrêt AUTO,25.000000


In [7]:

qry0 = "SELECT * FROM `box_stats` WHERE type LIKE 'dirtying' and box_id= (SELECT id FROM boxes WHERE serialNumber LIKE '%37cf0') ORDER BY `box_stats`.`date` ASC "
df = read_query(conn,qry0)
df0 =read_query(conn,qry0)

fig = px.line(df0, x="date", y="value", color="type" )
fig.add_scatter(x=dff.date, y=dff['normalised_state'], mode='lines',name="filtration",marker_color='rgba(0, 0, 0, .9)')
#fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(255, 0, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True  
        ),
        type="date"
    )
) 


fig.show()


In [8]:

qry1 = "SELECT * FROM `box_stats` WHERE type IN ('dirtying') ORDER BY `box_stats`.`date` ASC "
seklect * from box_stats where type in ('dirtying') order by date asc;

df1 =read_query(conn,qry1)

fig = px.line(df1.iloc[7:], x="date", y="value", color="box_id" )


fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True  
        ),
        type="date"
    )
) 


fig.show()


In [20]:
fig.write_html('encrassement.html')

In [21]:
#(SELECT id FROM boxes WHERE serialNumber LIKE '%37cf0')
#"SELECT distinct(box_id) FROM `box_stats` WHERE type IN ('dirtying') ORDER BY `box_stats`.`date` ASC "

qry2= "SELECT box_id, date FROM `box_stats` WHERE type IN ('dirtying') ORDER BY `box_stats`.`date` ASC "
df2 = read_query(conn,qry2)
df2.iloc[7:]


,box_id,date
7,182,2022-04-28 22:59:23
8,182,2022-04-30 13:14:22
9,182,2022-05-01 08:14:24
10,182,2022-05-02 15:00:00
11,182,2022-05-03 15:00:00
...,...,...
422,363,2022-09-14 00:46:31
423,129,2022-09-14 06:45:47
424,182,2022-09-14 14:31:19
425,182,2022-09-14 16:00:00


In [26]:
from tracemalloc import take_snapshot


THRESHOLD = 10
qry0 = "SELECT * FROM `box_stats` WHERE type IN ('dirtying') and box_id= (SELECT id FROM boxes WHERE serialNumber LIKE '%37cf0') ORDER BY `box_stats`.`date` ASC "
df = read_query(conn,qry0)
df0 =read_query(conn,qry0)
df_dirtying = df.iloc[:,3:5]

df_dirtying['threshold'] = THRESHOLD
df_dirtying['event'] = df_dirtying.value < df_dirtying.threshold
df_dirtying['event'] = np.where(df_dirtying['event']==True, 1 ,0)
#df_dirtying = df_dirtying.set_index('date')
df_dirtying


,date,value,threshold,event
0,2022-04-28 22:59:23,12.0,10,0
1,2022-04-30 13:14:22,12.0,10,0
2,2022-05-01 08:14:24,12.0,10,0
3,2022-05-02 15:00:00,12.0,10,0
4,2022-05-03 15:00:00,12.0,10,0
...,...,...,...,...
89,2022-09-12 07:00:56,14.0,10,0
90,2022-09-12 16:00:00,14.0,10,0
91,2022-09-13 16:00:00,14.0,10,0
92,2022-09-14 14:31:19,14.0,10,0


In [28]:
import plotly.express as px


fig = px.bar(df_dirtying, x=df_dirtying.date, y="value")
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['value'], mode='lines',name="values",marker_color='rgba(0, 0, 0, .9)')
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(255, 0, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()



In [15]:
from doctest import master
from email.errors import MissingHeaderBodySeparatorDefect
from logging import lastResort


df_dirtying_true = df_dirtying[df_dirtying.event == 1]
df_dirtying_true.head(5)


,date,value,threshold,event
10,2022-05-09 16:59:39,3.0,10,1
11,2022-05-10 09:59:37,3.0,10,1
12,2022-05-11 09:29:36,3.0,10,1
13,2022-05-12 07:29:39,3.0,10,1
14,2022-05-13 15:29:42,3.0,10,1
15,2022-05-14 17:00:00,3.0,10,1
16,2022-05-16 13:59:43,7.0,10,1
17,2022-05-16 17:00:00,7.0,10,1
21,2022-06-30 16:00:35,6.0,10,1
22,2022-07-01 15:30:32,6.0,10,1


In [87]:
import plotly.express as px

fig = px.line(df_dirtying, x=df_dirtying.date, y=df_dirtying.value) 
fig.add_scatter(x=df_dirtying_true.date, y=df_dirtying_true.value, mode='markers',name="nettoyage de cartouche")
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(20, 200, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()




In [16]:
df_dirtying['_start'] = df_dirtying.date.shift(1)
# not really necessary to explicitly call pd.to_timedelta
# but makes it clear what's happening
df_dirtying['_duration'] = pandas.to_timedelta(df_dirtying.date - df_dirtying._start)
print(f'ON:  {df_dirtying.loc[df_dirtying.event.eq(1)]._duration.sum()}')
#print(f'OFF: {df_dirtying_true.loc[df_dirtying_true.event.eq(0)]._duration.sum()}')


ON:  32 days 10:36:26


In [18]:
df_dirtying.head(60)

,date,value,threshold,event,_start,_duration
0,2022-04-28 22:59:23,12.0,10,0,NaT,NaT
1,2022-04-30 13:14:22,12.0,10,0,2022-04-28 22:59:23,1 days 14:14:59
2,2022-05-01 08:14:24,12.0,10,0,2022-04-30 13:14:22,0 days 19:00:02
3,2022-05-02 15:00:00,12.0,10,0,2022-05-01 08:14:24,1 days 06:45:36
4,2022-05-03 15:00:00,12.0,10,0,2022-05-02 15:00:00,1 days 00:00:00
5,2022-05-03 19:51:12,12.0,10,0,2022-05-03 15:00:00,0 days 04:51:12
6,2022-05-05 18:14:32,12.0,10,0,2022-05-03 19:51:12,1 days 22:23:20
7,2022-05-06 17:44:31,12.0,10,0,2022-05-05 18:14:32,0 days 23:29:59
8,2022-05-07 02:29:39,12.0,10,0,2022-05-06 17:44:31,0 days 08:45:08
9,2022-05-08 18:19:41,16.0,10,0,2022-05-07 02:29:39,1 days 15:50:02
